#### Exercise 1

In [ ]:
# These lines are just creating the right types of geometrical objects with the right coordinates
coords1 = [((5.5, 0), (5,5), (0, 6.5), (5, 9), (5.5, 14), (9, 10.5), (14, 12), (11.5, 6.5), (14, 2), (9, 3.5), (5.5, 0))]
coords2 = [((-10, -8), (-9,-9), (-8,-8), (-8, 0)), ((-8, 0), (-1,0), (-4,4), (-8,5), (-12,4), (-15, 0), (-8, 0))]
geom1 = shapely.geometry.MultiLineString(coords1)
geom2 = shapely.geometry.MultiLineString(coords2)

# These are the two lists mentioned in the prompt
list_geom    = [geom1, geom2]
list_strings = ['Starman', 'Singing in the Rain']

# Create a GeoDataFrame
new_gdf = gpd.GeoDataFrame({'Titles': list_strings , 'geometry': list_geom})

# plot it
new_gdf.plot()
# Rotate the geometry by 90 degress.
# For example, new_gdf.rotate(180) will rotate the geometry by 180 degrees
new_gdf['geometry'] = new_gdf['geometry'].rotate(90)
# plot the rotate geometry
new_gdf.plot()

# Get the area of the geometry
print(new_gdf.area)
# Why these values for the area?
# Because the objects in the geometry are lines. Lines have no area

#### Exercise 2

In [ ]:
# Beware that ALAND10 and AWATER10 are in m^2. We will use km^2.
# This lines do the conversion
blk['AWATER10'] = blk['AWATER10']*0.000001
blk['ALAND10']  = blk['ALAND10']*0.000001
# Density is numb. of people divided by total area.
# Construct the new column using the 'TOTAL' column at the numerator
blk['density']  = blk['TOTAL']/(blk['AWATER10']+blk['ALAND10']) 
# plot it
blk.plot(column = 'density', cmap = 'bone', 
         linewidth=0.02, edgecolor='red',
        )
# It turns out that the map is not very informative
# The problem is always the same: there is one block with sky high density
# Controlling the scale we get a much more informative view
blk.plot(column = 'density', cmap = 'bone', 
         linewidth=0.02, edgecolor='red',
         vmin = 0, vmax = 8000)

# Second part
# Calculate the area from GeoPandas
blk['area_crs0'] = blk.area
# Beware this is in feet squared. This line will convert it to km^2
blk['area_crs0'] = blk['area_crs0'] * 0.09290304 * 0.000001
# Calculate the new density
blk['density_crs0'] = blk['TOTAL']/(blk['area_crs0'])
# plot density again
blk.plot(column = 'density_crs0', cmap = 'bone', 
         linewidth=0.02, edgecolor='red',
         vmin = 0, vmax = 8000)
# Convert the crs EPSG:32616
blk = blk.to_crs(32616)
# Calculate the area again
blk['area_crs1'] = blk.area
# This time the area in m^2. Convert it to km^2
blk['area_crs1'] = blk['area_crs1']*0.000001
# Calculate the new density
blk['density_crs1'] = blk['TOTAL']/(blk['area_crs1'])
# plot it
blk.plot(column = 'density_crs1', cmap = 'bone', 
         linewidth=0.02, edgecolor='red',
         vmin = 0, vmax = 8000)
blk = blk.to_crs(epsg_numb)

# What is the difference?
# There are many ways to assess the difference.
# Here, I simply take the mean (of the absolute value) of the difference 
# between the GeoPandas-based densities and 
# the density based on the Census areas
mn_diff_crs0 = (blk['density_crs0'] - (blk['density'])).abs().mean()
print(f'This is the difference with the Illinois specific CRS:{mn_diff_crs0}')
mn_diff_crs1 = (blk['density_crs1'] - (blk['density'])).abs().mean()
print(f'This is the difference with the less specific CRS:{mn_diff_crs1}')
# We see the regional CRS is substantially closer to the 'true' value
# But, is this difference substantially meaningful? Depends on your questions!

#### Exercise 2

##### First block

In [ ]:
# To measure income segregation, we need income data
# Read in the income data. These files contains
# the number of households in Census pre-defined income brackets
# for each group of blocks in Cook County.
url1 = 'https://raw.githubusercontent.com/non87/GeoPandas-Workshop/master/data/grpblk/inc_2013.csv'
url2 = 'https://raw.githubusercontent.com/non87/GeoPandas-Workshop/master/data/grpblk/inc_2017.csv'
inc13 = pd.read_csv(url1)
inc17 = pd.read_csv(url2)
# As often the case with Census file, the column names are mysterious
# Use this dictionary along with the .rename( ) method to make things better
col_names_dic = {'GEO_ID': 'GEOID10'      , 'B19001_002E': '<10k'    , 
                 'B19001_003E': '10_15k'  , 'B19001_004E': '15_20k'  ,
                 'B19001_005E': '20_25k'  , 'B19001_006E': '25_30k'  ,
                 'B19001_007E': '30_35k'  , 'B19001_008E': '35_40k'  ,
                 'B19001_009E': '40_45k'  , 'B19001_010E': '45_50k'  ,
                 'B19001_011E': '50_60k'  , 'B19001_012E': '60_75k'  ,
                 'B19001_013E': '75_100k' , 'B19001_014E': '100_125k',
                 'B19001_015E': '125_150k', 'B19001_016E': '150_200k',
                 'B19001_017E': '>200k'   , 'B19001_001E': 'TOTAL'
                }
# Use rename and col_names_dic to rename the columns of the df
inc13 = inc13.rename(columns = col_names_dic)
inc17 = inc17.rename(columns = col_names_dic)
# Using the dic we save a list of columns we would like to retain.
# The other columns in the data are not relevant for us.
# We use a list comprehension for this
retain = [it for k, it in col_names_dic.items()]
# Drop those columns that are not in retain using a selection.
# This is just for our own clarity while working with data
inc13 = inc13.loc[:, retain]
inc17 = inc17.loc[:, retain]
# The first line of the df is troublesome
# This is another common feature of Census data.
# Delete it from both df using the .drop( ) method of dataframes
inc13 = inc13.drop(0)
inc17 = inc17.drop(0)
# Notice that the dtypes of the df are wrong
# This is due to the row we just deleted.
# This is a list containing the names of the columns to convert
conv_cols = [col for col in retain if col != 'GEOID10']
# Convert them to the right dtype (int32 will do)
for col in conv_cols:
    inc13[col] = inc13[col].astype('int32')
    inc17[col] = inc17[col].astype('int32')

##### Second block

In [ ]:
# These two variables contains the median income of 2013 and 2017
med13 = 54548
med17 = 59426
# Now we need to determine how many households have an income above the
# median in each group of blocks.

# First: sum the number of families that are certainly below the median
# You may find helpful to have this list of col names
brackets_below = ['<10k', '10_15k', '15_20k', '20_25k', '25_30k', '30_35k',
                  '35_40k',  '40_45k', '45_50k']
# Try using the .sum() method on a selection
inc13['below'] = inc13.loc[:, brackets_below].sum(axis = 1)
inc17['below'] = inc17.loc[:, brackets_below].sum(axis = 1)

# Here we encounter another common problem:
# the median falls within an income bracket and we don't know how
# to estimate the number of families who are above or below the median within this bracket
# Here we take the simplistic approach that income is uniformly
# distributed within income brackets. More sophisticated approaches exist.
bracket_within = ['50_60k']
# This is the simple estimate, rounded to nearest integer
approx13 = np.round(inc13[bracket_within].to_numpy()*((med13 - 50000)/10000))
approx13 = approx13.reshape((approx13.shape[0], ))
approx17 = np.round(inc17[bracket_within].to_numpy()*((med17 - 50000)/10000))
approx17 = approx17.reshape((approx17.shape[0], ))
# We add the estimate to the what we already calculated
inc13['below'] = inc13['below'] + approx13
inc17['below'] = inc17['below'] + approx17

# Finally, calculate the proportion of households below the median
# We add the year to the column name to facilitate the merging later
inc13['prop_below_13'] = inc13['below']/inc13['TOTAL']
inc17['prop_below_17'] = inc17['below']/inc17['TOTAL']
# Notice there will be NaNs in those blocks who have 0 population

# Before merging it is convenient to drop all columns we don't care about
# in our case, we only care about the proportion and GEOID10.
# Use a selection to drop the other columns
inc13 = inc13.loc[:, ['GEOID10', 'prop_below_13']]
inc17 = inc17.loc[:, ['GEOID10', 'prop_below_17']]

##### Third block

In [ ]:
# Use .merge( ) to merge the inc13 data with blk
blk = blk.merge(inc13, on = 'GEOID10')
# Same for inc17
blk = blk.merge(inc17, on = 'GEOID10')
print(blk.head())
# Plot the proportion below median income in 2013
# Use "vmin = 0, vmax = 1" to make sure the scale goes from 0 to 1
blk.plot(column = 'prop_below_13', cmap = 'gist_gray_r', 
         linewidth=0.2, edgecolor='black', vmin = 0, vmax = 1,
        legend = True)
# Plot the proportion below median income in 2017
# As before, "vmin = 0, vmax = 1" to make sure the scale goes from 0 to 1
blk.plot(column = 'prop_below_17', cmap = 'gist_gray_r', 
         linewidth=0.2, edgecolor='black', vmin = 0, vmax = 1,
        legend = True)

# A little extra, plot the difference between the proportions in the two years
# to see change more clearly. Beware, we need to use a diverging cmap
blk['prop_diff'] = blk['prop_below_17'] - blk['prop_below_13']
blk.plot(column = 'prop_diff', cmap = 'RdGy', vmin = -1, vmax = 1,
         linewidth=0.2, edgecolor='black', legend = True)